#### Kim Vo - Lab 1
This project will convert each feature in a shapefile to GeoJSON file.

In [1]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

In [2]:
# Initialize work env and declare overwrite output
arcpy.env.workspace = "U:\\GEOG458\\NewSHP\\"
arcpy.env.overwriteOutput = True

My first approach for this step was to join the table and the shapefile together, but the AddJoin_management in arcpy took very long time to run, so I decided to add new field to store the county name. 

In [3]:
# Create new field for county names
arcpy.AddField_management("saep_bg10.shp", "cntyName", "TEXT")

# Loop through table to get county name
nameList = []
curS = arcpy.da.SearchCursor("WashingtonFIPS.dbf", ["CountyName", "FIPSCounty"])
for row in curS:
    nameList.append(row[0])
    # Update county name for new field in the shp
    curU = arcpy.da.UpdateCursor("saep_bg10.shp", "cntyName", '"COUNTYFP10" = ' + "'%s'" %row[1])
                                # Use where_clause to query out blocks with given county name
    for x in curU:
        x[0] = row[0]
        curU.updateRow(x)
del curS
del curU

In [11]:
# Set up the environment variables for GDAL/OGR
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"

For this step, I use where clause to query out all blocks belong to each county by county name, and export them into 39 shapefiles

In [13]:
# Export shapefiles for each county
for name in nameList:
    arcpy.FeatureClassToFeatureClass_conversion("saep_bg10.shp", "U:\\GEOG458\\NewSHP\\ExportSHP\\",
                                                "" + name, '"cntyName" = ' + "'%s'" %name)
                                                # Use where_clause to query out blocks with given county name

I used to provided resource to find the projection which is 2927, and then converted them to 39 geojson files

In [14]:
# Convert each shp into geoJSON file
for name in nameList:
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      'U:\\GEOG458\\NewSHP\\ExportJSON\\' + name + '.geojson',
      'U:\\GEOG458\\NewSHP\\ExportSHP\\' + name + '.shp'])

I created a list to store population for each county, then sort the list in descending order, then I used while loop to print out the first 10.

In [4]:
# Add total population from each county to a list
popList = []
for name in nameList:
    pop = 0
    curS = arcpy.da.SearchCursor("saep_bg10.shp", ["POP2017"], '"cntyName" = ' + "'%s'" %name)
                                # Use where_clause to query out blocks with given county name
    for row in curS:
        pop += row[0]
    popList.append(pop)
    
# Sort popList into descending order
popList.sort(reverse=True)

# Print out population in descending order
i = 0
while i < 10:
    print popList[i]
    i += 1


2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


This project took me about 7 hours, because my first approach was wrong because I used dissolve to clean up my shapefile. It also took me awhile to realize AddJoin_management function doesn't work well in arcpy, so I have to switch to another method. For overall, I did learned a lot from this project, and it requires a lot of problem solving. 